In [59]:
from datetime import date

import numpy as np
import bermuda as tri
import altair as alt
alt.renderers.enable("html")

RendererRegistry.enable('html')

In [2]:
big_triangle = tri.binary_to_triangle("data/plotting_triangle.trib")

In [3]:
big_triangle.plot_data_completeness()

alt.Chart(...)

In [4]:
triangle = big_triangle.clip(min_period=date(2010, 1, 1))

In [5]:
triangle.plot_data_completeness()

alt.Chart(...)

In [6]:
triangle.plot_heatmap()

alt.LayerChart(...)

In [7]:
triangle.plot_heatmap("Reported Loss Ratio")

alt.LayerChart(...)

In [8]:
triangle.plot_heatmap("Reported ATA")

alt.LayerChart(...)

In [9]:
triangle.plot_atas()

alt.LayerChart(...)

In [10]:
triangle.plot_right_edge()

alt.LayerChart(...)

In [11]:
triangle.plot_mountain()

alt.LayerChart(...)

In [12]:
triangle.plot_hose()

alt.LayerChart(...)

In [13]:
triangle.plot_ballistic()

alt.LayerChart(...)

# Multi-Slice Plots

Suppose you're an insurer covering three lines of business, Commercial Auto, General Liability, and Private Auto. You'll likely have three separate triangles for each, and you might like to view each of them separately, or combine them all together. A single triangle object can have multiple "slices", or cuts by metadata such as line of business.

In [16]:
ca = tri.binary_to_triangle("data/ca.trib")
gl = tri.binary_to_triangle("data/gl.trib")
pa = tri.binary_to_triangle("data/pa.trib")

combined = pa + ca + gl

In [17]:
combined.plot_data_completeness()

alt.ConcatChart(...)

# Customizing Plots
The plots returned by these funcions are Altair LayerChart objects. You can manipulate these objects to customize them to suit your needs.

In [21]:
mountain = triangle.plot_mountain()
mountain

alt.LayerChart(...)

In [38]:
mountain.background = 'palegoldenrod'
mountain.title = "New Custom Name"
mountain.padding = {"left":40}
mountain

alt.LayerChart(...)

In [67]:
triangle.plot_mountain().properties(title="Other Method")

alt.LayerChart(...)

In [68]:
triangle.plot_right_edge().configure(
    background="#eeeeee",
 ).configure_title(
    font="monospace"
 ).configure_axisX(
    titleColor="blue",
    labelColor="green",
    tickColor="red",
    labelFontSize=10,
).configure_legend(
    direction="horizontal",
    orient="bottom",
).configure_bar(
    stroke="black",
)

alt.LayerChart(...)

# Plotting Stochastic Predictions

In [63]:
rng = np.random.default_rng(1234)
prediction_evals = triangle.evaluation_dates[-3:]
triangle_predictions = triangle.derive_fields(
    paid_loss = lambda cell: (
       rng.normal(cell["paid_loss"], 1e6 * cell.dev_lag(), size=1000)
       if cell.evaluation_date in prediction_evals
       else cell["paid_loss"]
    ),
    reported_loss = lambda cell: (
       rng.normal(cell["reported_loss"], 1e6 * cell.dev_lag(), size=1000)
       if cell.evaluation_date in prediction_evals
       else cell["reported_loss"]
    ),
)

triangle_predictions.plot_right_edge()

alt.LayerChart(...)

In [64]:
triangle_predictions.plot_right_edge(uncertainty_type="segments")

alt.LayerChart(...)

In [65]:
triangle_predictions.plot_growth_curve()

alt.LayerChart(...)

In [66]:
triangle_predictions.derive_metadata(
    id = lambda cell: cell.period_start
).plot_growth_curve(width=250, height=200, ncols=2)

alt.ConcatChart(...)

In [69]:
triangle_predictions.derive_metadata(
    id = lambda cell: cell.period_start
).plot_growth_curve(
    width=150, height=100, ncols=5,
    facet_titles=[str(period[0].year) for period in triangle_predictions.periods],
    uncertainty_type="spaghetti",
).resolve_scale(
    x="shared", y="shared", color="independent",
).resolve_axis(
    y="shared"
).properties(
    background="#eeeeee",
).configure_concat(
    spacing=2,
).configure_title(
    offset=1,
).interactive()

alt.ConcatChart(...)

In [71]:
triangle.plot_growth_curve(metric_spec = {"Reserves" : lambda cell: cell['reported_loss'] - cell['paid_loss']})

alt.LayerChart(...)

# Try it yourself!

### Problem 1
Try plotting the same reserves plot above, but show reserves as a percentage of earned premium on the y axis.

### Answer 1

In [80]:
triangle.plot_growth_curve(metric_spec = {"Reserves as % of Earned Premium" : lambda cell: 100 * (cell['reported_loss'] - cell['paid_loss']) / cell['earned_premium']})

alt.LayerChart(...)

### Problem 2

Now show the same reserves as a percentage of earned premium in a heatmap

### Answer 2

In [79]:
triangle.plot_heatmap(metric_spec = {"Reserves as % of Earned Premium" : lambda cell: 100 * (cell['reported_loss'] - cell['paid_loss']) / cell['earned_premium']})

alt.LayerChart(...)